# [IAPR][iapr]: Final project - Chocolate Recognition


**Moodle group ID:** 50  
**Kaggle challenge:** Classic  
**Kaggle team name (exact):** "ChocoBlabla"  

**Author 1 (SCIPER):** *Louis Cuendet (xxxxxx)*  
**Author 2 (SCIPER):** *Timur Ünver (xxxxx)*  
**Author 3 (SCIPER):** *Adrien Boschung (327221)*  

**Due date:** 21.05.2025 (11:59 pm)


## Key Submission Guidelines:
- **Before submitting your notebook, <span style="color:red;">rerun</span> it from scratch!** Go to: `Kernel` > `Restart & Run All`
- **Only groups of three will be accepted**, except in exceptional circumstances.


[iapr]: https://github.com/LTS5/iapr2025

---

## Segmentation

## Classification

In this section, we will detail which chocolate descriptors we decide to keep or not for classification and the reasons why.

The first important note is that the descriptors were computed on the reference images. We did not take into account other chocolates from the train dataset as we developed segmentation and classification in parallel and it was therefore easier to develop classification regarding the reference images that were much easier to segment. An improvement to our classification would be to average the chocolate descriptors accross the train set which would most probably yield a better result on the test set.


Our first idea was to first clusterize the chocolates in different categorize related to their shapes. To do so we decided to compute the Fourier descriptors as they worked well during the labs. 

In [ ]:
# Contours
contours_patterns = []

def contour_of_patterns(patterns):
    contours_patterns = []
    for i in range(len(patterns)):
        if i < n_images:
            plt.figure()
            img = patterns[i]
            img = np.array(img)
            img = np.mean(img, axis=2)
            binary = img > 0 
            contours = find_contours(binary, level=0.5)
            if contours:
                contour = np.fliplr(max(contours, key=lambda x: x.shape[0]))
                contours_patterns.append(contour)
                plt.imshow(patterns[i], cmap='gray')
                plt.axis('off')
                plt.plot(contour[:, 0], contour[:, 1], linewidth=2, color='red')  # use [:, 0] and [:, 1]
        else:
            ax.axis('off')  # Hide empty subplots

    return contours_patterns

contours_patterns = contour_of_patterns(patterns)

In [ ]:
def compute_descriptor_padding(contours: np.ndarray, n_samples: int = 9):
    """
    Compute Fourier descriptors of input images
    
    Args
    ----
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    n_samples: int
        Number of samples to consider. If the contour length is higher, discard the remaining part. If it is shorter, add padding.
        Make sure that the first element of the descriptor represents the continuous component.

    Return
    ------
    descriptors: np.ndarray complex (N, n_samples)
        Computed complex Fourier descriptors for the given input images
    """

    N = len(contours)
    # Look for the number of contours
    descriptors = np.zeros((N, n_samples), dtype=np.complex128)

    # ------------------
    for i in range(N):
        contour = contours[i]
        if len(contour) > n_samples:
            contour = contour[:n_samples]
        elif len(contour) < n_samples:
            contour = np.concatenate((contour, np.zeros((n_samples - len(contour), 2))), axis=0)
        descriptors[i, :] = np.fft.fft(contour[:, 0] + 1j * contour[:, 1], n_samples, axis=0)
    # ------------------

    return descriptors

def linear_interpolation(contours: np.ndarray, n_samples: int = 9):
    """
    Perform interpolation/resampling of the contour across n_samples.
    
    Args
    ----
    contours: list of np.ndarray
        List of N arrays containing the coordinates of the contour. Each element of the 
        list is an array of 2d coordinates (K, 2) where K depends on the number of elements 
        that form the contour. 
    n_samples: int
        Number of samples to consider along the contour.

    Return
    ------
    contours_inter: np.ndarray (N, n_samples, 2)
        Interpolated contour with n_samples
    """

    N = len(contours)
    contours_inter = np.zeros((N, n_samples, 2))
    
    # ------------------
    for i in range(N):
        # Get the contour to process
        contour = contours[i]
        # Compute the distance between points
        dist = np.sqrt(np.sum(np.diff(contour, axis=0) ** 2, axis=1))
        # Compute the cumulative distance
        cum_dist = np.concatenate(([0], np.cumsum(dist))) #with initial point at 0
        # Interpolate the points
        dist = np.linspace(0, cum_dist[-1], n_samples) #defining the distance axis (from 0 to the total distance at [-1] index)
        x_interp = np.interp(dist, cum_dist, contour[:, 0]) #x as a function of distance
        y_interp = np.interp(dist, cum_dist, contour[:, 1]) #y as a function of distance
        contours_inter[i] = np.array([x_interp, 
                                      y_interp]).T #transposing to get the right shape
    # ------------------
        
    return contours_inter

In [ ]:
def translation_invariant(features):
    """
    Make input Fourier descriptors invariant to translation.

    Args
    ----
    features: np.ndarray (N, D)
        The Fourier descriptors of N images over D features.

    Return
    ------
    features_inv: np.ndarray (N, K)
        The Fourier descriptors invariant to translation of N images 
        over K (K <= N) features.
    """

    # Set default values
    features_inv = np.zeros_like(features)
    
    # ------------------
    features_inv[:,1:] = features[:,1:]
    # ------------------
    
    return features_inv

In [ ]:
def rotation_invariant(features):
    """
    Make input Fourier descriptors invariant to rotation.

    Args
    ----
    features: np.ndarray (N, D)
        The Fourier descriptors of N images over D features.

    Return
    ------
    features_inv: np.ndarray (N, K)
        The Fourier descriptors invariant to rotation of N images 
        over K (K <= N) features.
    """

    # Set default values
    features_inv = np.zeros_like(features)
    
    # ------------------
    features_inv = np.abs(features)
    # ------------------
    
    return features_inv

In [ ]:
def normalize_descriptors(descriptors):
    normalized = []
    for des in descriptors:
        # Remove the DC component (translation invariance)
        des[0] = 0
        # Scale invariance
        des = des / np.abs(des[1])
        # Use only magnitudes (rotation invariance)
        normalized.append(np.abs(des))
    return np.array(normalized)

normalized_descriptors = normalize_descriptors(descriptors)

In [ ]:
# For the different shapes we average the descriptors
correct_labels = [0,1,1,1,0,0,1,2,3,3,2]
print(correct_labels)

In [ ]:
correct_labels = np.array(correct_labels)
unique_classes = np.unique(correct_labels)
mean_descriptors = {}

# Compute the mean descriptors for each class
for class_label in unique_classes:
    indices = np.where(correct_labels == class_label)[0]
    mean_descriptors[class_label] = np.mean(normalized_descriptors[indices], axis=0)

# Print the mean descriptors for each class
for class_label, mean_descriptor in mean_descriptors.items():
    print(f"Class {class_label}: Mean Descriptor")
    print(mean_descriptor)

Now we will plot the mean Fourier descriptors:

In [ ]:
plt.figure()
for d in mean_descriptors:
    plt.plot(mean_descriptors[d])

We see that the descriptors are very similar are therefore very hard to differentiate. They worked better in the labs as the numbers we were trying to retrieve were very different but in our case, the shape are similar and irregularities/inconsistencies in segmentation can easily lead in differences in the Fourier descriptor that will prevent us from correctly classifying the chocolate. For these reasons we decided to drop Fourier descriptors and focus on other features that are more insightful.

To further show you the similarities between the descriptors, we computed the how similar shapes are relative to each other (with)

In [ ]:
import numpy as np
import cv2

def to_cv2_contour(contour):
    return contour.reshape((-1, 1, 2)).astype(np.float32)

# Convert all contours to OpenCV format
cv2_contours = [to_cv2_contour(c) for c in contours_patterns]

# Create similarity matrix
n = len(cv2_contours)
similarity_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        similarity_matrix[i, j] = cv2.matchShapes(cv2_contours[i], cv2_contours[j], cv2.CONTOURS_MATCH_I1, 0)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(similarity_matrix)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Optional: chocolate names for x/y labels
choco_names = [
    "Amandina", "Arabia", "Comtesse", "Crème Brulée", "Jelly Black", "Jelly Milk", "Jelly White",
     "Noblesse", "Noir authentique", "Passion au lait", "Stracciatella", "Tentation noir", "Triangolo"
]

plt.figure(figsize=(10, 8))
sns.heatmap(scaled, xticklabels=choco_names, yticklabels=choco_names, cmap='viridis', annot=True, fmt=".3f")
plt.title("Shape Similarity Matrix (Lower = More Similar)")
plt.xlabel("Chocolate Class")
plt.ylabel("Chocolate Class")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

We see that chocolates are close to each other except for amandina and triangolo. For amandina there is another feature that we will develop later which already strongly differentiate amandina from the rest. In general, we decided to not use the Fourier descriptors due to imperfections in segmention which negatively impacts the Fourier descriptors and their use for segmentation.

Even though shapes are not useful by themselves, other geometric features are. By investigating them we found a few that successfully cluster the chocolates in three different categories. The 1st one is the compacity:

In [ ]:
def find_clusters(contours_patterns) :
    areas = []
    perimeters = []
    ratio = []
    compacity = []
    
    for contour in contours_patterns:
        contour = contour.astype(np.float32)
        areas.append(cv2.contourArea(contour))
        perimeters.append(cv2.arcLength(contour, closed=True))
        ratio.append(cv2.contourArea(contour)/cv2.arcLength(contour, closed=True))
        compacity.append(cv2.contourArea(contour)**2/cv2.arcLength(contour, closed=True))
        #print(f'Area = {area}, Perimeter = {perimeter}')

    """ plt.figure()
    plt.plot(range(len(perimeters)), perimeters, '.')
    plt.axhline(y=1750, color='green', linestyle='--')
    plt.axhline(y=1500, color='green', linestyle='--')
    plt.axhline(y=2200, color='green', linestyle='--') """

    plt.figure()
    x_pos = [1] * len(perimeters)  # All x-values are 1
    plt.plot(x_pos, perimeters, 'o')  # Use 'o' for point markers
    #plt.plot(1750, color='green')
    plt.xlabel('Samples')
    plt.ylabel('Perimeter')
    plt.title('Perimeters on a single vertical line')
    plt.show()

    """ plt.figure()
    plt.plot(range(len(areas)), areas, '.') """

    plt.figure()
    x_pos = [1] * len(areas)  # All x-values are 1
    plt.plot(x_pos, areas, 'o')  # Use 'o' for point markers
    #plt.plot(1750, color='green')
    plt.xlabel('Samples')
    plt.ylabel('Area')
    plt.title('Area on a single vertical line')
    plt.show()

    plt.figure()
    x_pos = [1] * len(ratio)  # All x-values are 1
    plt.plot(x_pos, ratio, 'o')  # Use 'o' for point markers
    #plt.axhline(y=102, color='red', linestyle='--', label='Threshold 1')
    #plt.axhline(y=121, color='green', linestyle='--', label='Threshold 2')
    plt.xlabel('Samples')
    plt.ylabel('Ratio')
    plt.title('Ratio on a single vertical line')
    plt.legend()
    plt.show()

    plt.figure()
    x_pos = [1] * len(compacity)  # All x-values are 1
    plt.plot(x_pos, compacity, 'o')  # Use 'o' for point markers
    #plt.axhline(y=100, color='red', linestyle='--', label='Threshold 1')
    #plt.axhline(y=120, color='green', linestyle='--', label='Threshold 2')
    plt.xlabel('Samples')
    plt.ylabel('Compacity')
    plt.title('Compacity on a single vertical line')
    plt.legend()
    plt.show()

    """ 
    plt.figure()
    plt.plot(areas, perimeters, '.') """
    return areas, perimeters, ratio, compacity

areas, perimeters, ratio, compacity = find_clusters(contours_patterns)

We see on the compacity plot (and area/perimeter) that we can extract two clusters. Chocolates with a compacity higher and lower than approximately 320'000. The 3 low compacity chocolates are the 3 Jelly's (Cluster 2). This is a good cluster as we can easily differentiate the Jelly chocolates by their RGB colors which we will show later.
Now we want to investigate if we can find another feature to further separate the bigger cluster of 10 chocolates. 

In [ ]:
cluster1 = []
cluster2 = []
for i in range(len(ratio)):
    if compacity[i] > 320000 :
        cluster1.append(i)
        continue
    else :
        cluster2.append(i)

print(cluster1)
print(cluster2)

NameError: name 'ratio' is not defined

In [ ]:
areas1, perimeters1, ratio1, compacity1 = find_clusters([contours_patterns[i] for i in cluster1])

On these graphs specific to the first cluster we see another feature that can clusterize the chocolates which is the ratio of the area over the perimeter. It separates high ratio chocolates (Cluster 1.1 : Crème Brulée, Noir authentique and Passion au lait) from the low ratio chocolates (CLuster 1.2 : Amandina, Arabia, Comtesse, Noblesse, Straciatella and Triangolo). These cluster seem good as each chocolates is quite different from each other.

We will of course color features (RGB) as well as HSV because they are basic features that have different values for different chocolates.

For each cluster, we build a different KNN classifier with the features extracted from the reference images.

#### CLuster 1.1

This cluster contains Crème Brulée, Noir authentique and Passion au lait.

For this cluster we decided to use the following features: 
- RGB 
- HSV 
- Contrast
- Stripe detection
- Perimeter
- Area
- Circularity

For this cluster, we observed surprising behavior. The performance on the train set was the highest when the only features used to differentiate them was the RGB values (no HSV nor contrast nor stripe detection) and then we tried to add perimeter, area and circularity which slightly improved performance. This is surprising, especially for HSV as the 3 chocolates have very different HSV values.

When fine tuning the classification, we observed that sometimes Triangolo's would be clusterize in 1.1 instead of 1.2. When adding Triangolo to 1.1 (and keeping it in 1.2 at the same time) we obtained better results. We do not think it is an issue to have a chocolate in both clusters as the other descriptors are different and will not lead to major misclassifications.

#### CLuster 1.2

This cluster contains Amandina, Arabia, Comtesse, Noblesse, Straciatella, Tentation noir and Triangolo.

For this cluster we decided to use the following features: 
- RGB 
- HSV 
- Texture
- Rectangularity
- Contrast
- Stripe detection

Stripe detection was used to classify Straciatella's as it contains white stripes. When classifying it in the KNN it led to some misclassification with Tentation noir and Arabia (but they have no stripes). This improved our performance on the train set. 

The rectangularity feature was used to try to differentiate the triangolo and tentation noir from the circular chocolates. We wanted looked into ways of quantifying triangularity such as peaks detection but it was hard to extract specific values for triangolo out of them (segmentation and other issues).

Contrast results in a wide range of values for the chocolates and was therefore deemed as a performant feature for the KNN.

#### CLuster 2

This cluster contains Jelly White, Jelly Milk and Jelly Black.

For this cluster we decided to use the following features: 
- RGB 
- HSV 
- Contrast

Jelly's all have the same shape so geometrical features will not be relevant in this case.

On the other hand they all have distinct colors so RGB and HSV are almost sufficient to classify them.

We also added contrast ...

#### Implementations for special cases

Black boxes and magnets : 

In the images there are some objects (black items and magnets) that have a similar size and shape as the chocolates that we want to identify. To try to recognize them and ignore them we sampled their RGB values in the train set and if the the object we are trying to classify has an RGB too close to these, we ignore it. This improved our performance significantly.

Smoothing chocolates :

It oftens happens that part of the shadow is included in the cropped out chocolate. In this case smoothing (more specifically opening) the cropped out chocolate can help to erase this unwanted feature and help correctly classify the chocolates. It also improved performance.

In [ ]:
# Apply opening to smoothen the contour
footprint = disk(15)
smoothed_mask = opening(mask_crop, footprint=footprint)
isolated_img = np.zeros_like(img_crop)
for c in range(3):
    isolated_img[..., c] = img_crop[..., c] * smoothed_mask

Watershed :

This algorithm is used to separate chocolates that are glued together. It can sometime fail to separate the chocolates in specific cases. For examples if two Triangolo's are standing next to each other side to side and not peak to peak it will fail to separate them and identify them successfully.

In [ ]:
# Large object: apply Watershed to split it
region_crop = isolated_mask[minr:maxr, minc:maxc].astype(np.uint8)
chocolate_crop = segmented_image[minr:maxr, minc:maxc]

# Create markers using distance transform
dist = cv2.distanceTransform(region_crop * 255, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist, 0.9 * dist.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(region_crop * 255, sure_fg)

# Marker labelling
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Watershed
chocolate_crop_color = chocolate_crop.copy()
markers = cv2.watershed(chocolate_crop_color, markers)

# Loop over each separated region (label > 1)
isolated_imgs = []
for label_val in np.unique(markers):
    if label_val <= 1:
        continue  # Skip background and unknown

    mask_ws = (markers == label_val).astype(np.uint8)
    if np.sum(mask_ws) < 10000:
        continue  # Skip tiny blobs

    # Create masked chocolate
    masked_choco = np.zeros_like(chocolate_crop_color)
    for c in range(3):
        masked_choco[..., c] = chocolate_crop_color[..., c] * mask_ws

    isolated_imgs.append(masked_choco)

Various Thresholds :

It happens that objects segmented cannot be chocolates if they are too large or too small. In those cases we have thresholds to ignore too large and too small objects. There is a special thresholding for objects that potentially correspond to 2 chocolates sticked together.

#### Abandoned ideas

Weights : 

An idea we had was to weight the different features depending on how much they contribute to the chocolate classification. To our surprise the performance on the train set was better when not weighting the features so we abandoned this idea.

Random forest

Mahalanobis distance